In [292]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

# Set pandas to display numbers with commas
pd.options.display.float_format = '{:,}'.format

# Prep Data and Create DB

### Import and Clean Data

In [293]:
# import and clean data

pluto_df = pd.read_csv("pluto.csv")
pluto_df.sort_values(by="zipcode", inplace=True)
pluto_df = pluto_df.reset_index(drop=True).copy()
pluto_df.dropna(inplace=True)
pluto_df.head()

,zipcode,borough,borocode,landuse,bldgarea,lotarea
0,"10,001.0",MN,1.0,5.0,"611,625.0","23,601.0"
1,"10,001.0",MN,1.0,4.0,"13,489.0","2,469.0"
2,"10,001.0",MN,1.0,5.0,"34,000.0","17,773.0"
3,"10,001.0",MN,1.0,11.0,0.0,0.0
4,"10,001.0",MN,1.0,5.0,"2,008.0","1,980.0"


In [294]:
#One hot encoding for landuse

dummy_df = pd.get_dummies(pluto_df.landuse)
pluto_df.drop("landuse", axis=1, inplace=True)
data_df = pluto_df.join(dummy_df)
data_df.head()

,zipcode,borough,borocode,bldgarea,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,"10,001.0",MN,1.0,"611,625.0","23,601.0",0,0,0,0,1,0,0,0,0,0,0
1,"10,001.0",MN,1.0,"13,489.0","2,469.0",0,0,0,1,0,0,0,0,0,0,0
2,"10,001.0",MN,1.0,"34,000.0","17,773.0",0,0,0,0,1,0,0,0,0,0,0
3,"10,001.0",MN,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1
4,"10,001.0",MN,1.0,"2,008.0","1,980.0",0,0,0,0,1,0,0,0,0,0,0


In [295]:
# Create column with dummy values that can be counted during groupby
# These will be used later to cull mini zipcodes in Manhattan with only a couple of buildings
data_df["bldg_count"] = data_df["zipcode"]

In [296]:
'''
Map bldgarea and lotarea to individual landuses
This gets confusing, but landuses associated with buildings get bldgarea and open space uses like parks get lot area
Effectively, this means that building uses and open space uses are in different units
But we can normalize each column individually for KNN

But before we even get that far, we need to groupby and sum for each zipcode, and then normalize those by total lot area
This ensures that large zipcodes don't appear more dense than small zipcodes
THEN we normalize each column for KNN to account for the differences btw building uses and open space uses
Two step normalization (NOT as clean or elegant as it should be...)
'''

# Assign parcel areas to each land use
cols = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
for col in cols:
    if col < 9:
        data_df[col] = data_df[col] * data_df.bldgarea
    else:
        data_df[col] = data_df[col] *data_df.lotarea

In [297]:
data_df

,zipcode,borough,borocode,bldgarea,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,bldg_count
0,"10,001.0",MN,1.0,"611,625.0","23,601.0",0.0,0.0,0.0,0.0,"611,625.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
1,"10,001.0",MN,1.0,"13,489.0","2,469.0",0.0,0.0,0.0,"13,489.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
2,"10,001.0",MN,1.0,"34,000.0","17,773.0",0.0,0.0,0.0,0.0,"34,000.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
3,"10,001.0",MN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
4,"10,001.0",MN,1.0,"2,008.0","1,980.0",0.0,0.0,0.0,0.0,"2,008.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
5,"10,001.0",MN,1.0,"58,764.0","4,937.0",0.0,0.0,0.0,0.0,0.0,"58,764.0",0.0,0.0,0.0,0.0,0.0,"10,001.0"
6,"10,001.0",MN,1.0,"113,000.0","7,407.0",0.0,0.0,0.0,0.0,0.0,"113,000.0",0.0,0.0,0.0,0.0,0.0,"10,001.0"
7,"10,001.0",MN,1.0,"13,000.0","2,364.0",0.0,0.0,"13,000.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,001.0"
8,"10,001.0",MN,1.0,"34,213.0","6,300.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"6,300.0",0.0,"10,001.0"
9,"10,001.0",MN,1.0,0.0,"8,636.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"8,636.0",0.0,"10,001.0"


In [298]:
data_df[data_df.zipcode == 10282]

,zipcode,borough,borocode,bldgarea,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,bldg_count
42048,"10,282.0",MN,1.0,"502,000.0","42,342.0",0.0,0.0,0.0,0.0,"502,000.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42049,"10,282.0",MN,1.0,"278,142.0","14,451.0",0.0,0.0,"278,142.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42050,"10,282.0",MN,1.0,"105,177.0","28,206.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"105,177.0",0.0,0.0,0.0,"10,282.0"
42051,"10,282.0",MN,1.0,"244,617.0","6,323.0",0.0,0.0,"244,617.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42052,"10,282.0",MN,1.0,"628,200.0","79,338.0",0.0,0.0,0.0,0.0,"628,200.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42053,"10,282.0",MN,1.0,"356,786.0","16,653.0",0.0,0.0,"356,786.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42054,"10,282.0",MN,1.0,"2,152,863.0","97,076.0",0.0,0.0,0.0,0.0,"2,152,863.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42055,"10,282.0",MN,1.0,"357,000.0","10,292.0",0.0,0.0,"357,000.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42056,"10,282.0",MN,1.0,"289,251.0","14,500.0",0.0,0.0,"289,251.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42058,"10,282.0",MN,1.0,"233,010.0","17,353.0",0.0,0.0,0.0,"233,010.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"


In [299]:
data_df.drop(42060, axis=0, inplace=True)

In [300]:
data_df[data_df.zipcode == 10282]

,zipcode,borough,borocode,bldgarea,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,bldg_count
42048,"10,282.0",MN,1.0,"502,000.0","42,342.0",0.0,0.0,0.0,0.0,"502,000.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42049,"10,282.0",MN,1.0,"278,142.0","14,451.0",0.0,0.0,"278,142.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42050,"10,282.0",MN,1.0,"105,177.0","28,206.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"105,177.0",0.0,0.0,0.0,"10,282.0"
42051,"10,282.0",MN,1.0,"244,617.0","6,323.0",0.0,0.0,"244,617.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42052,"10,282.0",MN,1.0,"628,200.0","79,338.0",0.0,0.0,0.0,0.0,"628,200.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42053,"10,282.0",MN,1.0,"356,786.0","16,653.0",0.0,0.0,"356,786.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42054,"10,282.0",MN,1.0,"2,152,863.0","97,076.0",0.0,0.0,0.0,0.0,"2,152,863.0",0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42055,"10,282.0",MN,1.0,"357,000.0","10,292.0",0.0,0.0,"357,000.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42056,"10,282.0",MN,1.0,"289,251.0","14,500.0",0.0,0.0,"289,251.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"
42058,"10,282.0",MN,1.0,"233,010.0","17,353.0",0.0,0.0,0.0,"233,010.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,282.0"


In [301]:
data_df.sort_values(by='lotarea')

,zipcode,borough,borocode,bldgarea,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,bldg_count
550489,"11,354.0",QN,4.0,"640,355.0",0.0,0.0,0.0,0.0,"640,355.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"11,354.0"
449652,"11,226.0",BK,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"11,226.0"
16419,"10,019.0",MN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,019.0"
76809,"10,305.0",SI,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,305.0"
146817,"10,314.0",SI,5.0,"1,482.0",0.0,"1,482.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,314.0"
388598,"11,217.0",BK,3.0,"3,745.0",0.0,"3,745.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"11,217.0"
568054,"11,357.0",QN,4.0,"2,980.0",0.0,"2,980.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"11,357.0"
95584,"10,307.0",SI,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,307.0"
76811,"10,305.0",SI,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,305.0"
84827,"10,306.0",SI,5.0,"2,210.0",0.0,"2,210.0",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,306.0"


In [302]:
# Groupby to create summary data

data_df.drop("borough", axis=1, inplace=True)
landuse_df = data_df.groupby("zipcode").agg({'borocode':'median',
                                             'lotarea': 'sum',
                                             1.0:'sum',
                                             2.0:'sum',
                                             3.0:'sum',
                                             4.0:'sum', 
                                             5.0:'sum',
                                             6.0:'sum',
                                             7.0:'sum',
                                             8.0:'sum',
                                             9.0:'sum',
                                             10.0:'sum',
                                             11.0:'sum',
                                             'bldg_count': 'count'
                                            })
landuse_df.reset_index(inplace=True)
landuse_df.head()

,zipcode,borocode,lotarea,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,bldg_count
0,"10,001.0",1.0,"12,931,894.0","33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","502,515.0","507,583.0","1,006,828.0",980
1,"10,002.0",1.0,"18,127,897.0","51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","4,397,528.0","185,666.0","513,831.0",1672
2,"10,003.0",1.0,"9,648,834.0","515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","426,087.0","88,655.0","47,221.0",1864
3,"10,004.0",1.0,"12,027,534.0",0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","1,083,714.0","11,398.0","15,493.0",112
4,"10,005.0",1.0,"1,934,277.0",0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,"2,877.0",64


In [303]:
# Relabel borocode so easy to remember (had to drop above b/c groupby only accepts numeric columns)
boro_dict = {"Manhattan": 1.0, 
             "Bronx": 2.0, 
             "Brooklyn": 3.0, 
             "Queens": 4.0, 
             "Staten Island": 5.0}

def relabel(num):
    boros = list(boro_dict.keys())
    values = list(boro_dict.values())    
    return boros[values.index(num)]

landuse_df['borocode'] = landuse_df['borocode'].apply(relabel)

In [304]:
# Read in and add neighborhoods (important for frontend display)
neighborhoods_df = pd.read_csv("neighborhoods.csv")
landuse_df = landuse_df.merge(neighborhoods_df, on="zipcode", how="inner")

In [305]:
# Rename colummns
cols_dict = {1.0: "1 & 2 Family Residential",
             2.0: "Multi-Family Walk-up",
             3.0: "Multi-Family Elevator",
             4.0: "Mixed Residential & Commercial",
             5.0: "Commercial & Office",
             6.0: "Industrial & Manufacturing",
             7.0: "Transportation & Utility",
             8.0: "Public Facilities & Institutions",
             9.0: "Open Space & Outdoor Recreation",
             10.0: "Parking",
             11.0: "Vacant Land"}

landuse_df.rename(columns = cols_dict, inplace=True)

In [306]:
# Reorder columns
cols = ['zipcode',
        'borocode',
        'neighborhood',
        'lotarea',
        'bldg_count',
        '1 & 2 Family Residential',
        'Multi-Family Walk-up',
        'Multi-Family Elevator',
        'Mixed Residential & Commercial',
        'Commercial & Office',
        'Industrial & Manufacturing',
        'Transportation & Utility',
        'Public Facilities & Institutions',
        'Open Space & Outdoor Recreation',
        'Parking',
        'Vacant Land']

landuse_df = landuse_df[cols]

In [307]:
# Remove miniscule zipcodes from Manhattan only (lots of 1-10 building mini-zipcodes)
landuse_df = landuse_df[(landuse_df.bldg_count >= 10) | (landuse_df.borocode != "Manhattan")].copy()
landuse_df.drop('bldg_count', axis=1, inplace=True)

In [308]:
landuse_df

,zipcode,borocode,neighborhood,lotarea,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
0,"10,001.0",Manhattan,NOMAD / Chelsea,"12,931,894.0","33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","502,515.0","507,583.0","1,006,828.0"
1,"10,002.0",Manhattan,Lower East Side,"18,127,897.0","51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","4,397,528.0","185,666.0","513,831.0"
2,"10,003.0",Manhattan,NOHO / Union Square,"9,648,834.0","515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","426,087.0","88,655.0","47,221.0"
3,"10,004.0",Manhattan,Bowling Green,"12,027,534.0",0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","1,083,714.0","11,398.0","15,493.0"
4,"10,005.0",Manhattan,Wall Street,"1,934,277.0",0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,"2,877.0"
5,"10,006.0",Manhattan,Lower Manhattan,"1,066,311.0",0.0,0.0,"2,406,484.0","1,409,933.0","7,224,839.0",0.0,0.0,"237,019.0","19,994.0","250,798.0","15,013.0"
6,"10,007.0",Manhattan,City Hall,"4,363,757.0","11,515.0","135,365.0","25,180,716.0","7,113,948.0","26,421,102.0","186,711.0","6,565.0","1,964,664.0","116,421.0","21,764.0","16,725.0"
7,"10,009.0",Manhattan,East Village,"11,715,549.0","49,533.0","4,551,976.0","7,867,831.0","16,437,543.0","1,094,334.0","132,152.0","175,000.0","1,778,954.0","811,185.0","35,020.0","150,996.0"
8,"10,010.0",Manhattan,Flat Iron / Gramercy,"6,480,598.0","35,514.0","671,530.0","7,188,028.0","14,478,911.0","16,348,068.0","316,637.0","65,745.0","5,288,022.0","554,573.0","55,381.0","14,715.0"
9,"10,011.0",Manhattan,Chelsea,"13,357,415.0","1,029,569.0","4,471,226.0","14,800,035.0","18,123,314.0","15,296,761.0","1,648,286.0","464,332.0","4,294,146.0","703,041.0","64,599.0","471,070.0"


In [309]:
landuse_df[landuse_df.zipcode == 10282]

,zipcode,borocode,neighborhood,lotarea,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
68,"10,282.0",Manhattan,Battery Park City,"525,645.0",0.0,0.0,"1,764,981.0","1,804,143.0","3,283,063.0",0.0,0.0,"555,177.0",0.0,0.0,0.0


### Normalize Data

In [310]:
normalized_df = landuse_df.copy()

In [311]:
# # Normalize by lot area

# normalized_df['1 & 2 Family Residential'] = normalized_df['1 & 2 Family Residential'] / normalized_df['lotarea']
# normalized_df['Multi-Family Walk-up'] = normalized_df['Multi-Family Walk-up'] / normalized_df['lotarea']
# normalized_df['Multi-Family Elevator'] = normalized_df['Multi-Family Elevator'] / normalized_df['lotarea']
# normalized_df['Mixed Residential & Commercial'] = normalized_df['Mixed Residential & Commercial'] / normalized_df['lotarea']
# normalized_df['Commercial & Office'] = normalized_df['Commercial & Office'] / normalized_df['lotarea']
# normalized_df['Industrial & Manufacturing'] = normalized_df['Industrial & Manufacturing'] / normalized_df['lotarea']
# normalized_df['Transportation & Utility'] = normalized_df['Transportation & Utility'] / normalized_df['lotarea']
# normalized_df['Public Facilities & Institutions'] = normalized_df['Public Facilities & Institutions'] / normalized_df['lotarea']
# normalized_df['Open Space & Outdoor Recreation'] = normalized_df['Open Space & Outdoor Recreation'] / normalized_df['lotarea']
# normalized_df['Parking'] = normalized_df['Parking'] / normalized_df['lotarea']
# normalized_df['Vacant Land'] = normalized_df['Vacant Land'] / normalized_df['lotarea']

In [312]:
normalized_df.drop("lotarea", axis=1, inplace=True)

In [313]:
# # Scale landuse columns
# # Can't use StandardScaler b/c disparity in Parks will create huge numbers that will bias KNN to that variable

# x = normalized_df.iloc[:, 3:]
# scaler = preprocessing.MinMaxScaler()
# x_scaled = scaler.fit_transform(x)

# normalized_df.iloc[:, 3:] = x_scaled

In [314]:
normalized_df.head()

,zipcode,borocode,neighborhood,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
0,"10,001.0",Manhattan,NOMAD / Chelsea,"33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","502,515.0","507,583.0","1,006,828.0"
1,"10,002.0",Manhattan,Lower East Side,"51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","4,397,528.0","185,666.0","513,831.0"
2,"10,003.0",Manhattan,NOHO / Union Square,"515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","426,087.0","88,655.0","47,221.0"
3,"10,004.0",Manhattan,Bowling Green,0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","1,083,714.0","11,398.0","15,493.0"
4,"10,005.0",Manhattan,Wall Street,0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,"2,877.0"


### Ensure matching zipcodes between DB and geojson

In [315]:
# geo_path = "../nearest_neighborhood/static/geojson/zipcodes.geojson"

# with open(geo_path) as f:
#     d = json.load(f)
#     pprint(d)

In [316]:
# # Create new df from zipcode GeoJSON

# geo_path = "../nearest_neighborhood/static/geojson/zipcodes.geojson"

# zips = []
# areas = []
# neighborhood = []
# ids = []

# with open(geo_path) as f:
#     d = json.load(f)
#     #pprint(d)
#     for obj in d['features']:
#         zips.append(obj['properties']['postalCode'])
#         areas.append(obj['properties']['Shape_Area'])
#         neighborhood.append(obj['properties']['PO_NAME'])
#         ids.append(obj['id'])
        
# data = {"zipcode": zips, "id": ids, "area": areas, "neighborhood": neighborhood}
# geo_df = pd.DataFrame.from_dict(data)

# geo_df.sort_values("zipcode", inplace=True)
# geo_df.reset_index(drop=True, inplace=True)

In [317]:
# print(geo_df.to_string())

In [318]:
# csv_path = "../nearest_neighborhood/static/js/available_zipcodes.csv"
# landuse_df.zipcode.to_csv(csv_path, indext=False)

In [319]:
# List of zipcodes without corresponding geoJSON need to be removed from database
drop_zips = [11249, 22222]

In [320]:
def check_zip(zipcode):
    if zipcode not in drop_zips:
        return True
    else:
        return False

In [321]:
normalized_df = normalized_df[normalized_df.zipcode.apply(check_zip)].copy()

In [322]:
normalized_df

,zipcode,borocode,neighborhood,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
0,"10,001.0",Manhattan,NOMAD / Chelsea,"33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","502,515.0","507,583.0","1,006,828.0"
1,"10,002.0",Manhattan,Lower East Side,"51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","4,397,528.0","185,666.0","513,831.0"
2,"10,003.0",Manhattan,NOHO / Union Square,"515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","426,087.0","88,655.0","47,221.0"
3,"10,004.0",Manhattan,Bowling Green,0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","1,083,714.0","11,398.0","15,493.0"
4,"10,005.0",Manhattan,Wall Street,0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,"2,877.0"
5,"10,006.0",Manhattan,Lower Manhattan,0.0,0.0,"2,406,484.0","1,409,933.0","7,224,839.0",0.0,0.0,"237,019.0","19,994.0","250,798.0","15,013.0"
6,"10,007.0",Manhattan,City Hall,"11,515.0","135,365.0","25,180,716.0","7,113,948.0","26,421,102.0","186,711.0","6,565.0","1,964,664.0","116,421.0","21,764.0","16,725.0"
7,"10,009.0",Manhattan,East Village,"49,533.0","4,551,976.0","7,867,831.0","16,437,543.0","1,094,334.0","132,152.0","175,000.0","1,778,954.0","811,185.0","35,020.0","150,996.0"
8,"10,010.0",Manhattan,Flat Iron / Gramercy,"35,514.0","671,530.0","7,188,028.0","14,478,911.0","16,348,068.0","316,637.0","65,745.0","5,288,022.0","554,573.0","55,381.0","14,715.0"
9,"10,011.0",Manhattan,Chelsea,"1,029,569.0","4,471,226.0","14,800,035.0","18,123,314.0","15,296,761.0","1,648,286.0","464,332.0","4,294,146.0","703,041.0","64,599.0","471,070.0"


### Create Database file for Flask app

In [323]:
database_path = "../zipcodeDB.sqlite"

In [324]:
# Create Engine
# will create sqlite file if it doesn't already exist
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [325]:
# Push data to database
normalized_df.to_sql(name="zip_table", con=conn, if_exists ="replace", index=False)

# KNN Algorith for Backend

## Run Initial Query And Find Nearest Neighbor

In [326]:
# Specify target zipcode

#zipcode = 11101 #Long Island City, QNS
#zipcode = 11104 #Sunnyside, QNS
#zipcode = 11355 #Flushing, QNS
#zipcode = 11201 #Brooklyn Heights, BK
#zipcode = 10304 #Todt Hill, SI
#zipcode = 10016 #Murray Hill
#zipcode = 10022 # Midtown East
zipcode = 10034 # Inwood

In [327]:
database_path = "../zipcodeDB.sqlite"

# Create Engine
# will create sqlite file if it doesn't already exist
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

# Query records to test that it works
data_df = pd.read_sql("SELECT * FROM zip_table", conn)

In [328]:
data_df.head()

,zipcode,borocode,neighborhood,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
0,"10,001.0",Manhattan,NOMAD / Chelsea,"33,606.0","592,289.0","8,350,224.0","13,875,477.0","53,005,101.0","4,445,633.0","1,182,594.0","10,895,555.0","502,515.0","507,583.0","1,006,828.0"
1,"10,002.0",Manhattan,Lower East Side,"51,117.0","1,938,379.0","16,618,352.0","16,356,822.0","3,200,967.0","1,182,700.0","590,077.0","4,618,645.0","4,397,528.0","185,666.0","513,831.0"
2,"10,003.0",Manhattan,NOHO / Union Square,"515,395.0","3,786,590.0","12,746,613.0","15,687,708.0","10,760,299.0","714,504.0","3,459.0","9,501,989.0","426,087.0","88,655.0","47,221.0"
3,"10,004.0",Manhattan,Bowling Green,0.0,0.0,"725,023.0","4,194,913.0","21,117,265.0","85,472.0","462,543.0","3,722,844.0","1,083,714.0","11,398.0","15,493.0"
4,"10,005.0",Manhattan,Wall Street,0.0,0.0,"2,480,663.0","5,683,041.0","16,515,733.0",0.0,"682,450.0","43,568.0",0.0,0.0,"2,877.0"


In [329]:
# Split DFs so that selected outer-borough zipcode can find nearest neighbor zip in Manhattan
manhattan_df = data_df[data_df.borocode == "Manhattan"]
outer_borough_df = data_df[data_df.borocode != "Manhattan"]

In [330]:
# Create dictObj
# This formatting is reuired for the D3 chart, and is (believe it or not) the cleanest solution I could come up with...
dictObj = {"manhattan": {"borough": "Manhattan",
                         "zipcode": None,
                         "neighborhood": None,
                         "values": {"0": {"color": "#FFDD80", 
                                          "GFA": None,
                                          "label": "1 & 2 Family Residential"},
                                    "1": {"color": "#ff9100",
                                          "GFA": None,
                                          "label": "Multi-Family Walk-up"},
                                    "2": {"color": "#bf360c",
                                          "GFA": None,
                                          "label": "Multi-Family Elevator"},
                                    "3": {"color": "#ff5252",
                                          "GFA": None,
                                          "label": "Mixed Residential & Commercial"},
                                    "4": {"color": "#c51162",
                                          "GFA": None,
                                          "label": "Commercial & Office"},
                                    "5": {"color": "#7b1fa2",
                                          "GFA": None,
                                          "label": "Industrial & Manufacturing"},
                                    "6": {"color": "#ba68c8",
                                          "GFA": None,
                                          "label": "Transportation & Utility"},
                                    "7": {"color": "#0d47a1",
                                          "GFA": None,
                                          "label": "Public Facilities & Institutions"},
                                    "8": {"color": "#00bfa5",
                                          "GFA": None,
                                          "label": "Open Space & Outdoor Recreation"},
                                    "9": {"color": "#607d8b",
                                          "GFA": None, 
                                          "label": "Parking"},
                                    "10": {"color": "#263238",
                                           "GFA": None,
                                           "label": "Vacant Land"}
                                   }
                        },
           "outer": {"borough": None,
                             "zipcode": None,
                             "neighborhood": None,
                             "values": {"0": {"color": "#FFDD80", 
                                              "GFA": None,
                                              "label": "1 & 2 Family Residential"},
                                        "1": {"color": "#ff9100",
                                              "GFA": None,
                                              "label": "Multi-Family Walk-up"},
                                        "2": {"color": "#bf360c",
                                              "GFA": None,
                                              "label": "Multi-Family Elevator"},
                                        "3": {"color": "#ff5252",
                                              "GFA": None,
                                              "label": "Mixed Residential & Commercial"},
                                        "4": {"color": "#c51162",
                                              "GFA": None,
                                              "label": "Commercial & Office"},
                                        "5": {"color": "#7b1fa2",
                                              "GFA": None,
                                              "label": "Industrial & Manufacturing"},
                                        "6": {"color": "#ba68c8",
                                              "GFA": None,
                                              "label": "Transportation & Utility"},
                                        "7": {"color": "#0d47a1",
                                              "GFA": None,
                                              "label": "Public Facilities & Institutions"},
                                        "8": {"color": "#00bfa5",
                                              "GFA": None,
                                              "label": "Open Space & Outdoor Recreation"},
                                        "9": {"color": "#607d8b",
                                              "GFA": None, 
                                              "label": "Parking"},
                                        "10": {"color": "#263238",
                                               "GFA": None,
                                               "label": "Vacant Land"}
                                   }
                        }
          }


In [331]:
target_borough = data_df.borocode[data_df.zipcode == zipcode].tolist()[0]
target_neighborhood = data_df.neighborhood[data_df.zipcode == zipcode].tolist()[0]

In [332]:
target_borough

'Manhattan'

In [333]:
target_neighborhood

'Inwood'

### Initial Query

In [334]:
if target_borough == 'Manhattan':
    # Pull out target zipcode from manhattan_df
    target_df = manhattan_df[manhattan_df.zipcode == zipcode]
    
    # Define zipcode, neighborhood
    dictObj["manhattan"]["zipcode"] = zipcode
    dictObj["manhattan"]["neighborhood"] = target_neighborhood
    
    for item in dictObj["manhattan"]["values"]:
        value_type = dictObj["manhattan"]["values"][item]["label"]
        dictObj["manhattan"]["values"][item]["GFA"] = target_df[value_type].tolist()[0]
    
    
else:
    # Pull out target zipcode from outer_borough_df
    target_df = outer_borough_df[outer_borough_df.zipcode == zipcode]
    
    # Define zipcode, neighborhood, and borough
    dictObj["outer"]["zipcode"] = int(zipcode)
    dictObj["outer"]["neighborhood"] = target_neighborhood
    dictObj["outer"]["borough"] = target_borough
    
    for item in dictObj["outer"]["values"]:
        value_type = dictObj["outer"]["values"][item]["label"]
        dictObj["outer"]["values"][item]["GFA"] = target_df[value_type].tolist()[0]

In [335]:
target_df

,zipcode,borocode,neighborhood,1 & 2 Family Residential,Multi-Family Walk-up,Multi-Family Elevator,Mixed Residential & Commercial,Commercial & Office,Industrial & Manufacturing,Transportation & Utility,Public Facilities & Institutions,Open Space & Outdoor Recreation,Parking,Vacant Land
31,"10,034.0",Manhattan,Inwood,"48,193.0","4,495,795.0","4,424,474.0","4,982,453.0","769,639.0","575,510.0","603,544.0","1,575,501.0","15,425,643.0","543,218.0","939,536.0"


In [336]:
# Either Manhattan or Outer Borough GFA should be empty
pprint(dictObj)

{'manhattan': {'borough': 'Manhattan',
               'neighborhood': 'Inwood',
               'values': {'0': {'GFA': 48193.0,
                                'color': '#FFDD80',
                                'label': '1 & 2 Family Residential'},
                          '1': {'GFA': 4495795.0,
                                'color': '#ff9100',
                                'label': 'Multi-Family Walk-up'},
                          '10': {'GFA': 939536.0,
                                 'color': '#263238',
                                 'label': 'Vacant Land'},
                          '2': {'GFA': 4424474.0,
                                'color': '#bf360c',
                                'label': 'Multi-Family Elevator'},
                          '3': {'GFA': 4982453.0,
                                'color': '#ff5252',
                                'label': 'Mixed Residential & Commercial'},
                          '4': {'GFA': 769639.0,
                         

### KNN

In [337]:
if target_borough == 'Manhattan': #i.e. the nearest neighbor will NOT be in Manhattan

    # Combine with Outerborough df to train model
    train_data = pd.concat([target_df, outer_borough_df])

    # Drop column with strings (b/c KNN needs numeric data) and reset index
    train_data.drop("borocode", axis=1, inplace=True)
    train_data.drop("neighborhood", axis=1, inplace=True)
    train_data.reset_index(drop=True, inplace=True)
    
    # Train Nearest Neighbors model
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(train_data)
    distances, indices = nbrs.kneighbors(train_data)

    # Find nearest neighbor
    nearest_neighbor_index = indices[0][1]
    result = train_data.iloc[nearest_neighbor_index, :]
    
    # Define zipcode, neighborhood, and borough
    result_zipcode = int(result["zipcode"])
    result_neighborhood = data_df[data_df.zipcode == result["zipcode"]].neighborhood.tolist()[0]
    result_borough = data_df[data_df.zipcode == result["zipcode"]].borocode.tolist()[0]
    dictObj["outer"]["zipcode"] = result_zipcode
    dictObj["outer"]["neighborhood"] = result_neighborhood
    dictObj["outer"]["borough"] = result_borough
    
    for item in dictObj["outer"]["values"]:
        value_type = dictObj["outer"]["values"][item]["label"]
        dictObj["outer"]["values"][item]["GFA"] = result[value_type]

        
else: # i.e. the nearest neighbor IS in Manhattan

    # Combine with Manhattan df to train model
    train_data = pd.concat([target_df, manhattan_df])
    
    # Drop column with strings and reset index
    train_data.drop("borocode", axis=1, inplace=True)
    train_data.drop("neighborhood", axis=1, inplace=True)
    train_data.reset_index(drop=True, inplace=True)
    
    # Train Nearest Neighbors model
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(train_data)
    distances, indices = nbrs.kneighbors(train_data)

    # Find nearest neighbor
    nearest_neighbor_index = indices[0][1]
    result = train_data.iloc[nearest_neighbor_index, :]
    
    # Define zipcode, neighborhood and borough
    result_zipcode = int(result["zipcode"])
    result_neighborhood = data_df[data_df.zipcode == result["zipcode"]].neighborhood.tolist()[0]
    result_borough = data_df[data_df.zipcode == result["zipcode"]].borocode.tolist()[0]
    dictObj["manhattan"]["zipcode"] = result_zipcode
    dictObj["manhattan"]["neighborhood"] = result_neighborhood
    dictObj["manhattan"]["borough"] = result_borough
    
    for item in dictObj["manhattan"]["values"]:
        value_type = dictObj["manhattan"]["values"][item]["label"]
        dictObj["manhattan"]["values"][item]["GFA"] = result[value_type]

In [338]:
pprint(dictObj)

{'manhattan': {'borough': 'Manhattan',
               'neighborhood': 'Inwood',
               'values': {'0': {'GFA': 48193.0,
                                'color': '#FFDD80',
                                'label': '1 & 2 Family Residential'},
                          '1': {'GFA': 4495795.0,
                                'color': '#ff9100',
                                'label': 'Multi-Family Walk-up'},
                          '10': {'GFA': 939536.0,
                                 'color': '#263238',
                                 'label': 'Vacant Land'},
                          '2': {'GFA': 4424474.0,
                                'color': '#bf360c',
                                'label': 'Multi-Family Elevator'},
                          '3': {'GFA': 4982453.0,
                                'color': '#ff5252',
                                'label': 'Mixed Residential & Commercial'},
                          '4': {'GFA': 769639.0,
                         